<a href="https://colab.research.google.com/github/danielbauer1979/FI830/blob/main/HestonPricer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**HESTON OPTION PRICING**

This codebook shows how to derive prices for simple vanilla European options given a set of parameters.

We make use of [quantlib](https://pypi.org/project/QuantLib/#description), a library for quantitative finance that is natively in C++ but can be pulled into Python. We load it in this Colab notebook via [pip](https://pip.pypa.io/en/stable/) the package istaller for Python.

In [1]:
!pip install QuantLib

     |████████████████████████████████| 17.9 MB 4.5 MB/s 


Now since Python is a general purpose programming language, we have to pull ib functionality from libraries. The most common ones for us are [pandas](https://pandas.pydata.org/), the data analysis library and math and [numpy](https://numpy.org/), which are mathematical libraries. And of course we have to import Quantlib, which we just installed (many packages such as pandas and numpy are natively installed in the Colab environment).

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime
from math import sqrt, exp
import QuantLib as ql

Now we are ready to define parameters. I use values from the literature. Note, for example, that theta = 0.035 for the variance implies that the mean reversion level (long-term value) for volatility is sqrt(0.035)=18.7%, which seems reasonable.

In [16]:
S_0 = 4456.18
v_0 = 0.2357*0.2357
kappa = 2.75
theta = 0.035
volvol = 0.425
rho = -0.4644
r = 0.0152
dividend = 0
today = ql.Date(23,3,2022)
day_count = ql.Actual365Fixed()

In [17]:
strike = 4500
maturity = ql.Date(16,3,2023)

Now to use the functionality of quantlib, we have to follow its defintions. This requires reading documentation and looking at examples. Gladly, I did that for you :-)

In [18]:
initial_value = ql.QuoteHandle(ql.SimpleQuote(S_0))
discount_curve = ql.YieldTermStructureHandle(ql.FlatForward(today,r,day_count))
dividend_yield = ql.YieldTermStructureHandle(ql.FlatForward(today,dividend,day_count))
heston_process = ql.HestonProcess(discount_curve,dividend_yield,initial_value,v_0,kappa,theta,volvol,rho)

In [26]:
call_payoff = ql.PlainVanillaPayoff(ql.Option.Call, strike) 
call_exercise = ql.EuropeanExercise(maturity)
option = ql.VanillaOption(call_payoff, call_exercise)
engine = ql.AnalyticHestonEngine(ql.HestonModel(heston_process),0.001,1000)
option.setPricingEngine(engine)
price = option.NPV()
print ("option_price", round(price,2))

option_price 354.64


...so that's the option price. Easy enough :-)

Let's also determine the option vega. Here there is some ambiguity, as the Heston model has variance as an input rather than volatility. So in our derivations, we showed that the relevant sensitivity/greek here is the sensitivity to the variance -- recall, we need to invest: 
$$
\frac{\frac{\partial V}{\partial v}}{\frac{\partial HedgingInstrument}{\partial v}}
$$
in the hedging instrument. To derive the sensitivity to the volatility, we can use the approximation from [here](https://quant.stackexchange.com/questions/60457/calculating-vega-in-heston).

In [32]:
bump = 0.0001
heston_process = ql.HestonProcess(discount_curve,dividend_yield,initial_value,v_0+bump,kappa,theta,volvol,rho)
engine = ql.AnalyticHestonEngine(ql.HestonModel(heston_process),0.001,1000)
option.setPricingEngine(engine)
price2 = option.NPV()
print ("option_price(bumped)", round(price2,2))
vega = (price2-price)/bump
print ("option_vega", round(vega,2))
vol_vega = vega * 2 * sqrt(v_0)
print ("option_vega(sigma)", round(vol_vega,2))

option_price(bumped) 354.78
option_vega 1456.53
option_vega(sigma) 686.61


Let's get the Black-Scholes implied volatility for that option.

In [7]:
sigma = ql.BlackVolTermStructureHandle(ql.BlackConstantVol(today, ql.TARGET(), 0.20, day_count))
process = ql.BlackScholesMertonProcess(initial_value,dividend_yield,discount_curve,sigma)

In [8]:
option.impliedVolatility(price, process)

0.19569801508385637